<a href="https://colab.research.google.com/github/Manal2005/Sentiment-Analysis/blob/main/Sentiment_Analysis_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q Kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"manal000","key":"4452dc291f10b8de99c2cab5cb3e1823"}'}

In [3]:
! kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('/content/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
import re
def remove_url(text):
    text = re.sub(r"http\S+", "", text)
    return text

In [8]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
def word_tokenizer(text):
    text = text.lower()
    text = text.split()

    return text

from nltk.corpus import stopwords

en_stopwords = set(stopwords.words('english'))

def remove_stopwords(text):
    text = [word for word in text if word not in en_stopwords]
    return text

import spacy
sp = spacy.load("en_core_web_sm")
def lemmatization(text):

    # text = [sp(word).lemma_ for word in text]

    text = " ".join(text)
    token = sp(text)

    text = [word.lemma_ for word in token]
    return text


import unicodedata as uni
def preprocessing(text):

    text = remove_url(text)
    text = uni.normalize('NFKD', text)
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = word_tokenizer(text)
    # text = stemming(text)
    text = lemmatization(text)
    text = remove_stopwords(text)
    text = " ".join(text)

    return text


In [10]:
from tqdm import tqdm

tqdm.pandas()

df['clean_review'] = df['review'].progress_map(preprocessing)


from tqdm import tqdm

tqdm.pandas()

df['clean_review2'] = df['clean_review'].progress_map(word_tokenizer)


100%|██████████| 50000/50000 [00:01<00:00, 27385.29it/s]


In [11]:
#Calculating Similarity Score
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api

# Load pre-trained Word2Vec model
model = api.load('word2vec-google-news-300')

# Example dictionary
category_dict = {
    'Cast': ['Acting', 'role', 'character', 'act', 'actress', 'actor', 'villain', 'protagonist', 'antagonist', 'performance', 'performed', 'play', 'played', 'playing', 'casting', 'cast', 'crew', 'artist', 'portray'],
    'Directing': ['Direct', 'directing', 'direction', 'filing', 'cinematography', 'filmmaker', 'cinematic', 'director'],
    'Plot': ['Plot', 'Story', 'Storyline', 'ending', 'storytelling', 'drama', 'writing', 'twist', 'script', 'writing', 'end', 'movie'],
    'Ambience': ['Scene', 'visual', 'effect', 'animation', 'cgi', 'graphic', 'scenary', 'stunt', 'design', 'audio', 'sound', 'music', 'track']
}

# Function to compute the average vector for a list of words
def average_vector(words, model):
    vectors = [model[word] for word in words if word in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Compute category embeddings
category_embeddings = {category: average_vector(words, model) for category, words in category_dict.items()}

# Example dataset
documents = df['clean_review2']


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [12]:
# Function to calculate similarity scores
def calculate_similarity_scores(documents, category_embeddings, model):
    results = []
    for doc in documents:
        doc_vector = average_vector(doc, model)
        similarities = {category: cosine_similarity([doc_vector], [embedding])[0][0] for category, embedding in category_embeddings.items()}
        results.append([doc] + list(similarities.values()))
    return results

# Calculate similarity scores for the dataset
output = calculate_similarity_scores(documents, category_embeddings, model)

# Convert to desired format

df2 = pd.DataFrame(output, columns=['Document', 'Cast_Score', 'Directing_Score', 'Plot_Score', 'Ambience_Score'])
df2.to_csv('output.csv', index=False)
df2


,Document,Cast_Score,Directing_Score,Plot_Score,Ambience_Score
0,"[one, reviewer, mention, watch, 1, oz, episode...",0.457709,0.289139,0.447424,0.497035
1,"[wonderful, little, production, br, br, film, ...",0.553624,0.406958,0.539988,0.621793
2,"[i, think, wonderful, way, spend, time, hot, s...",0.534575,0.325314,0.525913,0.515981
3,"[basically, family, little, boy, jake, think, ...",0.552289,0.439670,0.617834,0.515316
4,"[petter, matteis, love, time, money, visually,...",0.540026,0.375918,0.494050,0.540383
...,...,...,...,...,...
49995,"[i, think, movie, right, good, job, creative, ...",0.424374,0.325704,0.460989,0.504024
49996,"[bad, plot, bad, dialogue, bad, act, idiotic, ...",0.510556,0.328933,0.508185,0.555335
49997,"[i, catholic, teach, parochial, elementary, sc...",0.495764,0.307306,0.463895,0.432453
49998,"[i, go, disagree, previous, comment, side, mal...",0.479905,0.320995,0.465676,0.488828


In [17]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score

similarity_scores = df3[['Cast_Score', 'Directing_Score', 'Plot_Score', 'Ambience_Score']].values

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df3['Document'])
sequences = tokenizer.texts_to_sequences(df3['Document'])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df3['Sentiment'])  # Assuming 'positive' and 'negative' labels

# Combine text features with similarity scores
combined_features = np.hstack((padded_sequences, similarity_scores))

# Split the combined features into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels, test_size=0.2, random_state=42)

# Separate the padded_sequences and similarity_scores for training
X_train_text = X_train[:, :max_len]
X_train_similarity = X_train[:, max_len:]
X_test_text = X_test[:, :max_len]
X_test_similarity = X_test[:, max_len:]

# Define the LSTM model
embedding_dim = 100

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [18]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(X_train_text, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])


Epoch 1/10
1000/1000 [==============================] - 557s 553ms/step - loss: 0.3720 - accuracy: 0.8352 - val_loss: 0.3050 - val_accuracy: 0.8779
Epoch 2/10
1000/1000 [==============================] - 533s 533ms/step - loss: 0.2587 - accuracy: 0.8996 - val_loss: 0.3154 - val_accuracy: 0.8784
Epoch 3/10
1000/1000 [==============================] - 535s 535ms/step - loss: 0.2105 - accuracy: 0.9202 - val_loss: 0.3123 - val_accuracy: 0.8783
Epoch 4/10
1000/1000 [==============================] - 533s 533ms/step - loss: 0.1772 - accuracy: 0.9336 - val_loss: 0.3767 - val_accuracy: 0.8723


In [19]:
# Evaluate the model
y_pred_prob = model.predict(X_test_text)
y_pred = (y_pred_prob > 0.5).astype(int)

# Print classification report
print(classification_report(y_test, y_pred))

# Print accuracy separately
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')


313/313 [==============================] - 25s 77ms/step
              precision    recall  f1-score   support

           0       0.90      0.85      0.88      4961
           1       0.86      0.91      0.88      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

Accuracy: 0.8802
